In [1]:
from utils import AMAEConfig
from model import AudioMAE
import librosa
import os
import torch
from torchinfo import summary


In [7]:
test_cfg = AMAEConfig(cfg_path=r'E:\project\python\Audio_MAE_Base_ST\config\auto_encoder.yaml')
test_m = AudioMAE(test_cfg)
test_f = os.path.join('datas', 'test', 'test_10s.wav')
signal, _ = librosa.load(test_f, sr=32000, mono=True)
model_input = torch.Tensor(signal).unsqueeze(0)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\project\\python\\Audio_MAE_Base_ST\\config\\auto_encoder.yaml'

In [8]:
summary(test_m, input_size=(4, 320000), depth=5, col_names=("input_size", "output_size", "num_params", "trainable", "kernel_size"))

NameError: name 'test_m' is not defined

In [20]:
a = torch.tensor([2, 3, 5, 4, 1], dtype=torch.int32)
a = a.repeat(2, 4, 1).permute(0, 2, 1)
a.shape, a

(torch.Size([2, 5, 4]),
 tensor([[[2, 2, 2, 2],
          [3, 3, 3, 3],
          [5, 5, 5, 5],
          [4, 4, 4, 4],
          [1, 1, 1, 1]],
 
         [[2, 2, 2, 2],
          [3, 3, 3, 3],
          [5, 5, 5, 5],
          [4, 4, 4, 4],
          [1, 1, 1, 1]]], dtype=torch.int32))

In [28]:
B, L, C = a.shape
ori_mask_token = torch.zeros(1, 1, C)

len_keep = int(L * (1 - 0.6))
noise = torch.rand(B, L, device=a.device)  # noise in [0, 1]

# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=1)

# keep the first subset
ids_keep = ids_shuffle[:, :len_keep]
x_reserve = torch.gather(a, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, C))
mask_tokens = ori_mask_token.repeat(a.shape[0], ids_restore.shape[1] - x_reserve.shape[1], 1)
temp_x = torch.cat([x_reserve, mask_tokens], dim=1)
x_mask = torch.gather(temp_x, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, a.shape[2]))
x_mask

tensor([[[2., 2., 2., 2.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [4., 4., 4., 4.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [3., 3., 3., 3.],
         [0., 0., 0., 0.],
         [4., 4., 4., 4.],
         [0., 0., 0., 0.]]])

In [4]:
ids_shuffle

tensor([4, 0, 1, 3, 2])

In [5]:
ids_restore

tensor([1, 2, 4, 3, 0])

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
temp_df = pd.DataFrame(data=None, columns=["file", "type", "label"])
len(temp_df.index)

In [ ]:
temp_df.loc[len(temp_df.index)] = [os.path.join('datas', 'test', 'test_10s.wav'), 'train', -1]
temp_df.loc[len(temp_df.index)] = [os.path.join('datas', 'test', 'test_30s.wav'), 'eval', -1]

In [ ]:
temp_df.reset_index(drop=True, inplace=True)
temp_df

In [ ]:
temp_df.to_csv(os.path.join('workspace', 'test.csv'))

In [ ]:
rdf = pd.read_csv(os.path.join('workspace', 'test.csv'), index_col=0)

In [ ]:
rdf

In [ ]:
list(rdf['file'])

In [ ]:
list(rdf[rdf['type'] == 'train']['file'])

In [ ]:
a = 'aabccd.hdf5'
a

In [ ]:
str_list = list(a)
str_list

In [ ]:
n = 5
str_list.insert(-5, f'_{n}')
str_list

In [ ]:
def get_hdf5_handler(base_name):
    suffix = base_name.split('.')[-1]
    prefix = base_name[:-len(suffix) - 1]
    hdf5_files = [prefix + f'_{i}.' + suffix for i in range(2)]
    h5f_l = []
    return h5f_l

result = get_hdf5_handler('test.hdf5')
result

In [ ]:
result = get_hdf5_handler('test.hdf5')
result

In [ ]:
ll = [round(i * 119 / 10) for i in range(1, 1+10)]
ll

In [ ]:
import bisect
bisect.bisect_left(ll, 119)

In [ ]:
import os
import torch
import librosa
import torchaudio
test_f = os.path.join('datas', 'test', 'test_5s.wav')

In [ ]:
signal, sr = torchaudio.load(test_f)
signal_tensor = torchaudio.functional.resample(signal, sr, 32000)
signal_tensor = signal_tensor[:, :16000]
signal_tensor.mean()

In [ ]:
fbank1 = torchaudio.compliance.kaldi.fbank(signal_tensor, sample_frequency=32000, num_mel_bins=64, frame_length=25, frame_shift=10, use_energy=False, htk_compat=True, window_type='hanning')
fbank1.shape, fbank1

In [ ]:
fbank2_m = torchaudio.transforms.MelSpectrogram(
    sample_rate=32000,
    n_fft = 800,
    hop_length = 320,
    f_min = 20.0,
    n_mels = 64,
)
fbank2 = fbank2_m(signal_tensor)
log_offset = 1e-6
fbank2 = torch.log(fbank2 + log_offset).permute(0, 2, 1).squeeze(0)
fbank2.shape, fbank2

In [ ]:
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
spe_m = Spectrogram(n_fft=800, hop_length=320, win_length=800)
fbank3_m = LogmelFilterBank(sr=32000, n_fft=800, n_mels=64, fmin=20., top_db=None)
fbank3 = spe_m(signal_tensor)
fbank3 = fbank3_m(fbank3).squeeze(0).squeeze(0)
fbank3.shape, fbank3

In [ ]:
import numpy as np
def _pre_process(waveform):
    waveform = waveform - waveform.mean()
    idx = np.random.randint(len(waveform))
    rolled_waveform = np.roll(waveform, idx)
    mag = np.random.beta(10, 10) + 0.5
    waveform = rolled_waveform * mag

    return torch.Tensor(waveform)

In [ ]:
waveform, _ = librosa.load(test_f, sr=32000)
# waveform = _pre_process(waveform)
type(waveform), waveform.shape

In [ ]:
from utils import AMAEConfig
cfg = AMAEConfig(cfg_path='config/pretrain.yaml')

In [ ]:
print(cfg)

In [ ]:
import torch

In [ ]:
a = torch.tensor(list(range(12)), dtype=torch.int32)
a

In [ ]:
a = a.reshape(3, 4)
a

In [ ]:
a.permute(1, 0)

In [ ]:
def random_mask(self, x):
    B, C, T, F = x.shape
    L = T * F
    x = x.reshape(B, C, -1)

    len_keep = int(L * (1 - self.mask_ratio))
    noise = torch.rand(B, C, L, device=x.device)  # noise in [0, 1]
    ids_shuffle = torch.argsort(noise, dim=2)  # ascend: small is keep, large is remove
    ids_restore = torch.argsort(ids_shuffle, dim=2)

    keep_mask = torch.zeros([B, C, L], device=x.device)
    keep_mask[:, :, :len_keep] = 1
    keep_mask = torch.gather(keep_mask, dim=2, index=ids_restore)
    x_masked = torch.mul(x, keep_mask)

    remove_mask = torch.ones([B, C, L], device=x.device)
    if self.restore_mask_only:
        remove_mask[:, :, :len_keep] = 0
        remove_mask = torch.gather(remove_mask, dim=2, index=ids_restore)

    return x_masked.reshape(B, C, T, F), remove_mask.reshape(B, C, T, F)